In [ ]:
from pathlib import Path
import sys

def find_file_upwards(filename, start_path='.'):
    current_path = Path(start_path).resolve()

    for parent in [current_path] + list(current_path.parents):
        if (parent / filename).exists():
            return parent
    return None

ROOT_PATH = find_file_upwards("pyproject.toml")
sys.path.append(str(ROOT_PATH))

In [ ]:
from cahos import Scale, scale_schema, VoiceLeading, voice_leading_schema, get_scales, make_scale, make_voice_leading, \
    Instruments, mark_changes, get_swaps, get_span, get_n_pseudo_changes, count_upward_motion, count_downward_motion, midis_to_names, \
    calculate_entropy, calculate_sequence_entropy, calculate_motion_balance, get_max_step_size, IntSequence12
import polars as pl
import mido
from dataclasses import dataclass
from typing import Tuple
from operator import ge, le, and_, or_, eq
import numpy as np
from multiprocessing import Pool
from functools import reduce, partial
import random
from itertools import product, permutations
import time

In [ ]:
data_dir = Path(ROOT_PATH).joinpath("data")
data_dir.mkdir(exist_ok=True)

In [ ]:
ensemble = [
    Instruments.flute,
    Instruments.oboe,
    Instruments.clarinet,
    Instruments.bassoon,
    Instruments.horn,
    Instruments.horn,
    Instruments.trumpet,
    Instruments.violin,
    Instruments.violin,
    Instruments.viola,
    Instruments.cello,
    Instruments.contrabass
]
ensemble.sort(key=lambda inst: inst.register[0])

In [ ]:
saved_path = data_dir / "base_chords.parquet"

load_saved = True
if load_saved and saved_path.exists():
    base_chords = pl.read_parquet(saved_path)
else:
    max_span = 69
    allowed_intervals = [1, 2, 3, 4, 5, 6, 7]
    disallowed_beginnings = []
    disallowed_subsequences = [
        [1, 2], [2, 1], [3, 1], [1, 3],
        # [3, 4], [4, 3],
        # [6, 1], [1, 6], [5, 1], [1, 5],
        [1, 1], [2, 2],  # [3, 3], [4, 4],
        # [5, 5, 5], [6, 6, 6], [7, 7, 7]
    ]
    
    base_chords = pl.DataFrame(
        data=(s for s in get_scales(
            allowed_intervals=allowed_intervals,
            disallowed_subsequences=disallowed_subsequences,
            disallowed_beginnings=disallowed_beginnings,
            max_span=max_span)
        ),
        schema=scale_schema
    )
    
    base_chords.write_parquet(saved_path, compression="zstd")

print(len(base_chords))

In [ ]:
selected_chords = base_chords.filter(
    pl.col("n_unique_intervals") <= 3,
    # pl.col("span") >= 48,
    # pl.col("entropy") >= 1.5,
    # pl.col("sequence_entropy") >= 22.0
).sort(["sequence_entropy", "entropy", "span"], descending=True)

print(len(selected_chords))
selected_chords.unique(subset=["interval_set"], maintain_order=True)

In [ ]:
reals = np.cumsum(np.vstack(selected_chords["intervals"].to_numpy()), axis=1, dtype=int)
reals = np.hstack([
    np.zeros((reals.shape[0], 1), dtype=reals.dtype),
    reals
])

offsets = np.arange(*ensemble[0].register)
reals = reals[:, :, None] + offsets
reals = reals.transpose(0, 2, 1)
reals.shape

In [ ]:
indices = []
do_append = True
for i, chord_type in enumerate(reals):
    for j, based_chord in enumerate(chord_type):
        for idx, instrument in enumerate(ensemble):
            if based_chord[idx] < instrument.register[0] or based_chord[idx] > instrument.register[1]:
                do_append = False
                break
        if do_append:
            indices.append([i, j])
        do_append = True

indexing = np.array(indices)
selected = reals[indexing[:, 0], indexing[:, 1]]
selected.shape

In [ ]:
idx = np.argmin(selected[:, -1])
idx, selected[idx]

In [ ]:
selected[(selected[:, 0] == 31) & (selected[:, -1] == 64)]

In [ ]:
zeroed = selected - selected[:, :1]
jdx = np.argmin(zeroed[:, -1])
jdx, selected[jdx]

In [ ]:
selected[(selected[:, 0] == 34) & (selected[:, -1] == 65)]

In [ ]:
def new_make_voice_leading(
    midis_a: IntSequence12, midis_b: IntSequence12
) -> VoiceLeading:
    intervals_a = list(np.diff(midis_a))
    intervals_b = list(np.diff(midis_b))
    midis_a, midis_b = list(midis_a), list(midis_b)
    base_a = midis_a[0]
    base_b = midis_b[0]
    changes = mark_changes(midis_a, midis_b)
    common_notes = [a for a, b in zip(midis_a, midis_b) if a == b]
    changed_notes = [(a, b) for a, b in zip(midis_a, midis_b) if a != b]
    swaps = get_swaps(midis_a, midis_b)
    change_in_span = get_span(midis_b) - get_span(midis_a)
    n_upward_motion = count_upward_motion(changed_notes)
    n_downward_motion = count_downward_motion(changed_notes)

    return VoiceLeading(
        intervals_a=intervals_a,
        intervals_b=intervals_b,
        base_a=base_a,
        base_b=base_b,
        midis_a=midis_a,
        midis_b=midis_b,
        note_names_a=midis_to_names(midis_a),
        note_names_b=midis_to_names(midis_b),
        onehot_changed_voices=changes,
        entropy_of_changes=calculate_entropy(changes),
        sequence_entropy_of_changes=calculate_sequence_entropy(changes),
        common_notes=common_notes,
        changed_notes=changed_notes,
        swaps=swaps,
        n_changed_notes=len(changed_notes),
        n_common_notes=len(common_notes),
        n_swaps=len(swaps),
        change_in_span=change_in_span,
        n_upward_motion=n_upward_motion,
        n_downward_motion=n_downward_motion,
        motion_balance=calculate_motion_balance(n_upward_motion, n_downward_motion),
        max_step_size=get_max_step_size(changed_notes),
        n_pseudo_changes=get_n_pseudo_changes(midis_a, changed_notes),
    )

In [ ]:
rng = np.random.default_rng()

In [ ]:
def get_pairs_from_bass(a, b, preds, instrument=0):
    # Preapare random indices for shuffled iteration
    first = selected[(selected[:, instrument] == a) & (selected[:, -1] == 64)].copy()
    if b is not None:
        second = selected[selected[:, instrument] == b].copy()
    else:
        second = selected.copy()
    
    xs = np.arange(first.shape[0])
    rng.shuffle(xs)
    
    ys = np.arange(second.shape[0])
    rng.shuffle(ys)
    
    # loop through pairs
    for i in xs:
        for j in ys:
            # chord must change
            if np.all(first[i] == second[j]):
                continue
    
            vl = new_make_voice_leading(first[i], second[j])
            # return if all predicates are satisfied
            if reduce(and_, (func(vl) for func in preds), True):
                return vl

def get_pair_with_fixed_first(first, b, preds, instrument=0):
    if b is not None:
        second = selected[selected[:, instrument] == b].copy()
    else:
        second = selected.copy()
    ys = np.arange(second.shape[0])
    rng.shuffle(ys)

    for j in ys:
        # chord must change
        if np.all(first == second[j]):
            continue

        vl = new_make_voice_leading(first, second[j])
        # return if all predicates are satisfied
        if reduce(and_, (func(vl) for func in preds), True):
            return vl


def get_progression_from_bass_line(midis, preds, instrument=0):
    chord_pairs = []
    for idx in range(len(midis) - 1):
        if idx == 0:
            pair = get_pairs_from_bass(midis[idx], midis[idx+1], preds, instrument)
        else:
            pair = get_pair_with_fixed_first(np.array(pair.midis_b), midis[idx+1], preds, instrument)
        if pair is None:
            # raise ValueError(f"At index: {idx} (pitch pair: {(midis[idx], midis[idx+1])})")
            chord_pairs = chord_pairs[:-1]
            idx -= 1
        chord_pairs.append(pair)

    return pl.DataFrame(chord_pairs, schema=voice_leading_schema)


def get_progression_from_one_note(note, n, preds, instrument):
    chord_pairs = []
    idx = 0
    while idx < n:
        if idx == 0:
            pair = get_pairs_from_bass(note, None, preds, instrument)
        else:
            pair = get_pair_with_fixed_first(np.array(chord_pairs[-1].midis_b), None, preds, instrument)
        if pair is None:
            print(f"Back tracking... {idx=}")
            if idx == 0:
                print("Nothing found...")
                return []
            chord_pairs = chord_pairs[:-1]
            idx -= 1
            continue
            
        chord_pairs.append(pair)
        idx += 1

    return pl.DataFrame(chord_pairs, schema=voice_leading_schema)

In [ ]:
predicates = [
    lambda vl: vl.n_pseudo_changes == 0,
    # lambda vl: vl.n_common_notes == 7,
    lambda vl: vl.n_common_notes <= 5,
    lambda vl: vl.max_step_size <= 7,
    lambda vl: vl.n_swaps == 0,
    lambda vl: vl.motion_balance <= 4/5,
    lambda vl: vl.motion_balance >= 1/5,
    # lambda vl: vl.n_upward_motion >= 3,
    # lambda vl: vl.change_in_span >= 2,
    # lambda vl: vl.change_in_span >= 4
    # lambda vl: sum(vl.intervals_b) <= sum(vl.intervals_a),
    # lambda vl: vl.entropy_of_changes >= 0.9,
    lambda vl: vl.midis_b[0] >= vl.midis_a[0]
] 

progression = get_progression_from_one_note(31, 32, predicates, 0)
progression

In [ ]:
mid = mido.MidiFile()
mid.ticks_per_beat = 480
tracks = [mido.MidiTrack() for _ in range(12)]
for t in tracks:
    mid.tracks.append(t)

did_name = False
for entry in progression.iter_rows(named=True):
    if not did_name:
        for t, m, i in zip(mid.tracks, entry["midis_a"][::-1], ensemble[::-1]):
            t.append(mido.MetaMessage('track_name', name=i.name, time=0))
            t.append(mido.Message('program_change', program=i.program, time=0))
            did_name = True

            t.append(mido.Message('note_on', note=m, velocity=64, time=0))
            t.append(mido.Message('note_off', note=m, velocity=127, time=480))
    
    for t, m, i in zip(mid.tracks, entry["midis_b"][::-1], ensemble[::-1]):
        t.append(mido.Message('note_on', note=m, velocity=64, time=0))
        t.append(mido.Message('note_off', note=m, velocity=127, time=480))

out_dir = Path(ROOT_PATH).joinpath("out")

mid.save(out_dir / f'm_{round(time.time())}.mid')

# TODOs

- [ ] create a Phrase or Passage or Line or something class to store a string of chord motions
- [x] put in instrument ranges and track numbers, program change message to set instruments
- [ ] generate midi messages into appropriate tracks, some tracks may have multiple voices while others have none
- [x] put in a switch to treat bass as octave lower (midi 35 will be converted to midi 23) can also move all the others up an octave